<a href="https://colab.research.google.com/github/jisoolee11/study/blob/main/pytorch_tutorials/Learn_the_Basic_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization

## Pre-requisite Code

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


## Hyperparameters

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

Train Loop: 훈련 데이터셋을 반복하여 최적의 매개변수로 수렴

Validation/Test Loop: 모델 성능 확인을 위해 테스트 데이터셋을 반복

In [12]:
# train loop
# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# optimizer.zero_grad()
# loss.backwards()
# optimizer.step()

## Full Implementation

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------
loss: 2.207037 [    0/60000]
loss: 2.201976 [ 6400/60000]
loss: 2.165751 [12800/60000]
loss: 2.213141 [19200/60000]
loss: 2.169955 [25600/60000]
loss: 2.109786 [32000/60000]
loss: 2.179661 [38400/60000]
loss: 2.101140 [44800/60000]
loss: 2.110944 [51200/60000]
loss: 2.130359 [57600/60000]
Test Error: 
 Accuracy: 45.9%, Avg loss: 0.032673 

Epoch 2
--------------------------------
loss: 2.057757 [    0/60000]
loss: 2.049021 [ 6400/60000]
loss: 1.977344 [12800/60000]
loss: 2.096293 [19200/60000]
loss: 1.983731 [25600/60000]
loss: 1.891560 [32000/60000]
loss: 2.021235 [38400/60000]
loss: 1.877508 [44800/60000]
loss: 1.912287 [51200/60000]
loss: 1.926715 [57600/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 0.029197 

Epoch 3
--------------------------------
loss: 1.816680 [    0/60000]
loss: 1.803364 [ 6400/60000]
loss: 1.697848 [12800/60000]
loss: 1.894641 [19200/60000]
loss: 1.723706 [25600/60000]
loss: 1.614555 [32000/60000]
loss: 1.788447 [384